В качестве baseline берется случай, когда в качестве рекомендаций система выдает топ-k самых популярных исполнителей из тестового множества.

In [0]:
!pip install nptyping
!pip install ml_metrics

In [0]:
import sys
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import ml_metrics 
from nptyping import NDArray
from typing import Any

DATA_DIR = './drive/My Drive/Colab Notebooks/VK_internship/'
sys.path.append(DATA_DIR)
recommendation = __import__("recommendation") 
# Я работаю в среде google colaboratory, 
# поэтому так импортрую свой скрипт recommendation.py

In [0]:
user_artists_df = pd.read_csv(DATA_DIR+"users_artists.csv",                               
                              usecols = ["user_id", "artists"])
artists_id = user_artists_df["artists"].unique()
artists_id.sort()
# Так как в дальнейшем я хочу использовать идентификаторы исполнителей 
# в качестве индексов для матрицы смежности, то для train я беру первые 
# несколько последовательных идентификаторов:
train, test = train_test_split(artists_id, test_size=0.1)
del artists_id

Берется 21 самый популярный исполнитель, так как максимальная из рассматриваемых метрик MAP@20 и так как задача в поиске похожих исполнителей, то в рекомендациях должен отсутствовать исполнитель, для которого составляются рекомендации.

In [0]:
top_k_artists = recommendation.get_topk_artists_from_idxs(21, user_artists_df, train) 
# отсортированы по встречаемости

Далее идет подсчет всех метрик для элементов тестового множества.

In [0]:
metrics = {}
for k in [1,5,10,20]:
    metrics[k]=0

for artist_id in test:    
    # Поиск списков исполнителей, которых слушают те же пользователи, 
    # что и artist_id:
    users = np.unique(user_artists_df.loc[user_artists_df["artists"]==artist_id, "user_id"].values)
    similar_artists = user_artists_df.loc[user_artists_df["user_id"].isin(users), "artists"].drop_duplicates().values
    similar_artists = similar_artists[similar_artists!=artist_id]
    recommend = top_k_artists[top_k_artists!=artist_id]
    for key in metrics.keys():
        metrics[key] += ml_metrics.apk(
            actual=list(similar_artists),
            predicted=list(recommend[:key]),
            k=key
        )


In [0]:
for key in metrics.keys():
  metrics[key]/=len(test)
  print("MAP@{}: ".format(key), metrics[key])

MAP@1:  0.44686089454247024
MAP@5:  0.3633231073855081
MAP@10:  0.31999485833633884
MAP@20:  0.287371898637225


Результаты вполне неплохие для такого простого способа рекомендаций: почти в половине случаев самый популярный исполнитель тренировочного множества соответствует валидной рекомендации в тестовом множестве.